In [1]:
#ETL Libraries
import pandas as pd
import numpy as np

#Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

#To Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

#Statistical Packages
from scipy import stats
from scipy.stats import norm, skew

#Tar File Extraction
import tarfile

#Importing os package
import os
import shutil
from shutil import copyfile


#Audio Python Files
from os import path
from pydub import AudioSegment

#Importing Zip file
from zipfile import ZipFile

#Audio
import librosa

In [2]:
#Setting Paths
destination='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audio'
final_destination='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Final_Audio'

In [3]:
#Importing Transcript File
transcript = pd.read_csv("D:\\Toxicity-Detection-in-Spoken-Utterances\\Toxicity-Detection-in-Spoken-Utterances\\Data\\Transcripts.csv", encoding='ISO-8859-1')


## MELD

In [61]:
#Meld wav file path
source = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audio\train_splits"


In [62]:
#Getting List of MELD file names
selected_files=transcript[transcript['Dataset']=='MELD']['FileName'].unique()
selected_files=list(selected_files)

In [60]:
import os
import shutil

# Loop through the selected files and copy them to the destination
success_count = 0
failure_count = 0

print(f"\nCopying {len(selected_files)} files...")

# Create the destination directory if it doesn't exist
os.makedirs(destination, exist_ok=True)

for Filename in selected_files:
    try:
        # Remove the prefix (train_, dev_, test_)
        if '_' in Filename:
            core_file_name = Filename.split('_', 1)[-1]  # This will keep everything after the first underscore
        else:
            core_file_name = Filename  # No prefix to remove
            
        # Construct the full path to the MP4 file
        src_path = os.path.join(source, core_file_name + ".mp4")
        
        # Check if the source file exists
        if not os.path.exists(src_path):
            # Try alternative - maybe the file has the full name including prefix?
            alt_src_path = os.path.join(source, Filename + ".mp4")
            
            if os.path.exists(alt_src_path):
                src_path = alt_src_path
            else:
                print(f"❌ Source file not found: {core_file_name}.mp4")
                failure_count += 1
                continue
        
        # Define the destination path (keeping the MP4 extension for now)
        dst_path = os.path.join(destination, core_file_name + ".mp4")
        
        # Copy the file to destination
        shutil.copy2(src_path, dst_path)
        
        success_count += 1
        
        # Print progress every 20 files
        if success_count % 20 == 0:
            print(f"Progress: {success_count}/{len(selected_files)} files copied")
        
    except Exception as e:
        print(f"❌ Error copying {Filename}: {str(e)}")
        failure_count += 1

print(f"\nCopy operation completed!")
print(f"✓ Successfully copied: {success_count} files")
print(f"❌ Failed to copy: {failure_count} files")
print(f"\nNext steps:")
print(f"1. Install a dedicated audio converter like 'Audacity' or 'Format Factory'")
print(f"2. Use it to batch convert the MP4 files to WAV format with 16kHz, 16-bit, mono settings")


Copying 427 files...
❌ Source file not found: dia259_utt9.mp4
Progress: 20/427 files copied
❌ Source file not found: dia181_utt5.mp4
❌ Source file not found: dia156_utt15.mp4
Progress: 40/427 files copied
❌ Source file not found: dia230_utt7.mp4
Progress: 60/427 files copied
❌ Source file not found: dia123_utt15.mp4
Progress: 80/427 files copied
❌ Source file not found: dia140_utt6.mp4
Progress: 100/427 files copied
❌ Source file not found: dia196_utt15.mp4
Progress: 120/427 files copied
❌ Source file not found: dia166_utt7.mp4
❌ Source file not found: dia268_utt10.mp4
❌ Source file not found: dia246_utt8.mp4
Progress: 140/427 files copied
❌ Source file not found: dia203_utt11.mp4
❌ Source file not found: dia215_utt2.mp4
Progress: 160/427 files copied
❌ Source file not found: dia240_utt16.mp4
Progress: 180/427 files copied
❌ Source file not found: dia144_utt10.mp4
❌ Source file not found: dia125_utt18.mp4
❌ Source file not found: dia240_utt8.mp4
Progress: 200/427 files copied
❌ Source

In [63]:
import os
import subprocess
import glob

# Debug - show what's available in the source directory
print(f"Scanning directory: {source}")
files_in_source = glob.glob(os.path.join(source, "*.mp4"))
print(f"Found {len(files_in_source)} MP4 files in source directory")
if len(files_in_source) > 0:
    print(f"Sample files: {files_in_source[:5]}")

print(f"\nSelected files to process: {len(selected_files)}")
if len(selected_files) > 0:
    print(f"Sample selected files: {selected_files[:5]}")

# Check if ffmpeg is installed
try:
    subprocess.run(['ffmpeg', '-version'], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("✓ ffmpeg is installed and working")
except Exception as e:
    print(f"❌ ffmpeg is not installed or not in PATH: {e}")
    print("Please install ffmpeg and make sure it's in your system PATH")
    # Exit or handle appropriately

# Loop through the selected files and convert them to .wav format
for Filename in selected_files:
    try:
        # Remove the prefix (train_, dev_, test_)
        if '_' in Filename:
            core_file_name = Filename.split('_', 1)[-1]  # This will keep everything after the first underscore, e.g., 'dia557_utt7'
        else:
            core_file_name = Filename  # No prefix to remove
            
        # Print debug info
        print(f"\nProcessing: {Filename}")
        print(f"  Core filename: {core_file_name}")
        
        # Construct the full path to the MP4 file
        src_path = os.path.join(source, core_file_name + ".mp4")  # Using 'core_file_name' without the prefix
        print(f"  Looking for source file: {src_path}")
        
        # Check if the source file exists
        if not os.path.exists(src_path):
            # Try alternative - maybe the file has the full name including prefix?
            alt_src_path = os.path.join(source, Filename + ".mp4")
            print(f"  File not found. Trying alternative: {alt_src_path}")
            
            if os.path.exists(alt_src_path):
                src_path = alt_src_path
                print(f"  ✓ Found file with full name")
            else:
                print(f"❌ Source file does not exist: {src_path}")
                continue
        else:
            print(f"  ✓ Source file found")
        
        # Define the destination path with .wav extension
        dst_path = os.path.join(destination, core_file_name + ".wav")
        
        print(f"  Converting to: {dst_path}")
        
        # Use ffmpeg command line to convert the audio
        command = [
            'ffmpeg',
            '-i', src_path,
            '-acodec', 'pcm_s16le',
            '-ac', '1',
            '-ar', '16000',
            '-y',  # Overwrite output files
            dst_path
        ]
        
        try:
            result = subprocess.run(command, check=True, capture_output=True, text=True)
            print(f"✓ Successfully converted {Filename} to {core_file_name}.wav")
        except subprocess.CalledProcessError as e:
            print(f"❌ ffmpeg conversion failed: {e}")
            print(f"  Error output: {e.stderr}")
        
    except Exception as e:
        import traceback
        print(f"❌ Unexpected error processing {Filename}: {e}")
        print(traceback.format_exc())

print("\nConversion process completed!")


Scanning directory: D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audio\train_splits
Found 10422 MP4 files in source directory
Sample files: ['D:\\Toxicity-Detection-in-Spoken-Utterances\\Toxicity-Detection-in-Spoken-Utterances\\Audio\\train_splits\\dia0_utt0.mp4', 'D:\\Toxicity-Detection-in-Spoken-Utterances\\Toxicity-Detection-in-Spoken-Utterances\\Audio\\train_splits\\dia0_utt1.mp4', 'D:\\Toxicity-Detection-in-Spoken-Utterances\\Toxicity-Detection-in-Spoken-Utterances\\Audio\\train_splits\\dia0_utt10.mp4', 'D:\\Toxicity-Detection-in-Spoken-Utterances\\Toxicity-Detection-in-Spoken-Utterances\\Audio\\train_splits\\dia0_utt11.mp4', 'D:\\Toxicity-Detection-in-Spoken-Utterances\\Toxicity-Detection-in-Spoken-Utterances\\Audio\\train_splits\\dia0_utt12.mp4']

Selected files to process: 427
Sample selected files: ['test_dia259_utt9', 'test_dia13_utt5', 'train_dia557_utt7', 'train_dia761_utt9', 'train_dia424_utt12']
✓ ffmpeg is installed and working

Proc

In [64]:
import os
import shutil

# Define the source and destination directories
source_dir = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audio"
destination_dir = os.path.join(source_dir, "audio")

# Create the 'audio' folder if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# List all the files in the source directory
files = os.listdir(source_dir)

# Loop through the files and move .wav files to the 'audio' folder
for file in files:
    if file.endswith(".wav"):  # Check if the file is a .wav file
        src_path = os.path.join(source_dir, file)
        dst_path = os.path.join(destination_dir, file)

        # Move the .wav file to the new folder
        shutil.move(src_path, dst_path)
        print(f"Moved: {file}")

print("All .wav files have been moved to the 'audio' folder.")


Moved: dia1000_utt2.wav
Moved: dia1002_utt5.wav
Moved: dia1005_utt16.wav
Moved: dia1007_utt7.wav
Moved: dia1009_utt11.wav
Moved: dia1009_utt13.wav
Moved: dia1009_utt2.wav
Moved: dia1009_utt4.wav
Moved: dia1010_utt4.wav
Moved: dia1016_utt12.wav
Moved: dia1016_utt9.wav
Moved: dia1021_utt4.wav
Moved: dia1022_utt11.wav
Moved: dia1025_utt5.wav
Moved: dia102_utt2.wav
Moved: dia1034_utt1.wav
Moved: dia1036_utt15.wav
Moved: dia1038_utt1.wav
Moved: dia1038_utt4.wav
Moved: dia109_utt0.wav
Moved: dia111_utt1.wav
Moved: dia111_utt15.wav
Moved: dia111_utt4.wav
Moved: dia112_utt3.wav
Moved: dia115_utt3.wav
Moved: dia115_utt4.wav
Moved: dia119_utt3.wav
Moved: dia119_utt4.wav
Moved: dia11_utt6.wav
Moved: dia120_utt11.wav
Moved: dia120_utt7.wav
Moved: dia121_utt5.wav
Moved: dia122_utt0.wav
Moved: dia122_utt1.wav
Moved: dia126_utt5.wav
Moved: dia12_utt4.wav
Moved: dia130_utt10.wav
Moved: dia130_utt17.wav
Moved: dia138_utt10.wav
Moved: dia138_utt2.wav
Moved: dia139_utt3.wav
Moved: dia13_utt5.wav
Moved: d

## Common Voice

In [6]:
# Common Voice file Path
source='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audio'

In [7]:

#Opening Tar File
int_dest=source+'Audio Files - Required/'




In [8]:

#Getting List of Common Voice file names
selected_files=transcript[transcript['Dataset']=='Common Voice']['FileName'].unique()
selected_files=list(selected_files)


In [51]:
from tqdm import tqdm

file_names_set = set("cv-corpus-6.1-2020-12-11/en/clips/" + suit + ".mp3" for suit in selected_files)

members = [x for x in tar.getmembers() if x.name in file_names_set]

success_count = 0
fail_count = 0

for member in tqdm(members, desc="Extracting files"):
    try:
        tar.extract(member, path=int_dest)
        print(f"✅ Successfully extracted: {member.name}")
        success_count += 1
    except FileNotFoundError:
        print(f"❌ Failed to extract (Not found): {member.name}")
        fail_count += 1
    except Exception as e:
        print(f"⚠️ Error extracting {member.name}: {e}")
        fail_count += 1

print(f"\n🎉 Extraction finished. Success: {success_count}, Failed: {fail_count}")


Extracting files:   0%|                                                             | 3/8595 [00:02<1:34:49,  1.51it/s]

✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_100097.mp3
✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_10011.mp3
✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_100375.mp3
✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_100433.mp3
✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_100474.mp3
✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_100511.mp3


Extracting files:   0%|                                                               | 7/8595 [00:02<33:30,  4.27it/s]

✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_10067.mp3
✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_100714.mp3


Extracting files:   0%|                                                               | 9/8595 [00:02<28:14,  5.07it/s]

...

✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_98858.mp3
✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_98993.mp3
✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_99019.mp3
✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_99080.mp3


Extracting files: 100%|███████████████████████████████████████████████████████████▉| 8591/8595 [11:11<00:00, 15.95it/s]

✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_99259.mp3
✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_99506.mp3


Extracting files: 100%|████████████████████████████████████████████████████████████| 8595/8595 [11:11<00:00, 12.79it/s]

✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_9971.mp3
✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_99846.mp3
✅ Successfully extracted: cv-corpus-6.1-2020-12-11/en/clips/common_voice_en_9995.mp3

🎉 Extraction finished. Success: 8595, Failed: 0


In [9]:
#Setting new source path
source='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\AudioAudio Files - Required\cv-corpus-6.1-2020-12-11\en\clips'

In [10]:
from os import path
from pydub import AudioSegment

In [ ]:
import os
from pydub import AudioSegment
from tqdm import tqdm
import signal

# Define your input and output paths
# Assuming 'int_dest' and 'source' are already defined in your notebook
wav_output_dir = os.path.join(int_dest, "Wav Files")
os.makedirs(wav_output_dir, exist_ok=True)

# Timeout handler
class TimeoutException(Exception):
    pass

def handler(signum, frame):
    raise TimeoutException()

signal.signal(signal.SIGALRM, handler)  # Only works on Unix (Linux/macOS)

# Function to safely convert mp3 to wav
def convert_with_timeout(mp3_path, wav_path):
    try:
        signal.alarm(15)  # Timeout after 15 seconds
        sound = AudioSegment.from_mp3(mp3_path)
        sound.export(wav_path, format="wav")
        signal.alarm(0)  # Disable the alarm after successful execution
        return True
    except TimeoutException:
        print(f"⏰ Timeout while converting: {os.path.basename(mp3_path)}")
    except Exception as e:
        print(f"❌ Error converting {os.path.basename(mp3_path)}: {e}")
    return False

# Get all files in the source folder (limit to 8000)
files = os.listdir(source)[:8000]

# Loop with progress bar
for file in tqdm(files, desc="Converting MP3 to WAV"):
    try:
        if file.endswith(".mp3"):
            mp3_path = os.path.join(source, file)
            wav_path = os.path.join(wav_output_dir, file[:-4] + ".wav")  # Change extension

            success = convert_with_timeout(mp3_path, wav_path)
            if success:
                print(f"✅ Converted: {file}")
        else:
            print(f"⏭️ Skipped non-mp3 file: {file}")
    except Exception as e:
        print(f"🔥 Unexpected error with {file}: {e}")



In [ ]:
import os
from pydub import AudioSegment
from tqdm import tqdm
import multiprocessing
import signal

# Set your input directory and create output directory
input_dir = "int_dest"  # replace with your actual path
output_dir = os.path.join(input_dir, "wav_files")
os.makedirs(output_dir, exist_ok=True)

# Function to handle timeout
class TimeoutException(Exception):
    pass

def handler(signum, frame):
    raise TimeoutException()

signal.signal(signal.SIGALRM, handler)

# Conversion function with timeout
def convert_mp3_to_wav(mp3_path, wav_path):
    try:
        # 15 second timeout
        signal.alarm(15)
        sound = AudioSegment.from_mp3(mp3_path)
        sound.export(wav_path, format="wav")
        signal.alarm(0)
    except TimeoutException:
        print(f"Timeout while converting: {mp3_path}")
    except Exception as e:
        print(f"Error converting {mp3_path}: {e}")

# Gather all mp3 files (up to 8000)
mp3_files = [f for f in os.listdir(input_dir) if f.endswith('.mp3')][:8000]

# Main loop
for file in tqdm(mp3_files, desc="Converting MP3 to WAV"):
    try:
        mp3_path = os.path.join(input_dir, file)
        wav_filename = os.path.splitext(file)[0] + ".wav"
        wav_path = os.path.join(output_dir, wav_filename)
        convert_mp3_to_wav(mp3_path, wav_path)
    except Exception as e:
        print(f"Unexpected error with {file}: {e}")


In [11]:
import os
import shutil
from tqdm import tqdm

# List all files in the source WAV folder
files = os.listdir(int_dest + "Wav Files/")

# Move files with a progress bar
for file in tqdm(files, desc="Moving files to Detoxy Folder"):
    src_path = os.path.join(int_dest + "Wav Files", file)
    dst_path = os.path.join(destination, file)
    shutil.copy2(src_path, dst_path)


Moving files to Detoxy Folder: 100%|███████████████████████████████████████████████| 8595/8595 [03:19<00:00, 43.13it/s]


## LJ Speech

In [65]:
#Setting LJ Speech Path
source='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audio\wavs'

In [66]:
#Getting List of LJ Speech file names
selected_files=transcript[transcript['Dataset']=='LJ Speech']['FileName'].unique()
selected_files=list(selected_files)

In [68]:
#Moving Files to final location
import shutil

# Loop through the selected files and copy them to the destination folder
for i in range(len(selected_files)):
    # Ensure there's a backslash between the folder and the file
    src_path = os.path.join(source, selected_files[i] + ".wav")
    dst_path = os.path.join(destination, selected_files[i] + ".wav")

    # Copy the file to the destination
    shutil.copy2(src_path, dst_path)
    print(f"Successfully copied: {selected_files[i]}.wav")


Successfully copied: LJ036-0050.wav
Successfully copied: LJ006-0224.wav
Successfully copied: LJ042-0055.wav
Successfully copied: LJ017-0135.wav
Successfully copied: LJ002-0149.wav
Successfully copied: LJ040-0132.wav
Successfully copied: LJ003-0136.wav
Successfully copied: LJ003-0132.wav
Successfully copied: LJ028-0289.wav
Successfully copied: LJ030-0196.wav
Successfully copied: LJ045-0148.wav
Successfully copied: LJ037-0018.wav
Successfully copied: LJ041-0012.wav
Successfully copied: LJ018-0374.wav
Successfully copied: LJ017-0221.wav
Successfully copied: LJ008-0099.wav
Successfully copied: LJ013-0229.wav
Successfully copied: LJ014-0054.wav
Successfully copied: LJ010-0040.wav
Successfully copied: LJ016-0066.wav
Successfully copied: LJ018-0085.wav
Successfully copied: LJ015-0155.wav
Successfully copied: LJ016-0201.wav
Successfully copied: LJ009-0220.wav
Successfully copied: LJ014-0183.wav
Successfully copied: LJ014-0056.wav
Successfully copied: LJ014-0068.wav
Successfully copied: LJ041-0

In [69]:
import os
import shutil

# Define the source and destination directories
source_dir = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audio"
destination_dir = os.path.join(source_dir, "audio1")

# Create the 'audio' folder if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# List all the files in the source directory
files = os.listdir(source_dir)

# Loop through the files and move .wav files to the 'audio' folder
for file in files:
    if file.endswith(".wav"):  # Check if the file is a .wav file
        src_path = os.path.join(source_dir, file)
        dst_path = os.path.join(destination_dir, file)

        # Move the .wav file to the new folder
        shutil.move(src_path, dst_path)
        print(f"Moved: {file}")

print("All .wav files have been moved to the 'audio1' folder.")

Moved: LJ001-0092.wav
Moved: LJ002-0149.wav
Moved: LJ003-0132.wav
Moved: LJ003-0136.wav
Moved: LJ003-0330.wav
Moved: LJ004-0138.wav
Moved: LJ004-0169.wav
Moved: LJ005-0103.wav
Moved: LJ006-0057.wav
Moved: LJ006-0224.wav
Moved: LJ008-0066.wav
Moved: LJ008-0083.wav
Moved: LJ008-0099.wav
Moved: LJ008-0159.wav
Moved: LJ008-0161.wav
Moved: LJ008-0185.wav
Moved: LJ008-0251.wav
Moved: LJ009-0052.wav
Moved: LJ009-0054.wav
Moved: LJ009-0105.wav
Moved: LJ009-0220.wav
Moved: LJ010-0040.wav
Moved: LJ010-0048.wav
Moved: LJ010-0052.wav
Moved: LJ010-0053.wav
Moved: LJ010-0067.wav
Moved: LJ010-0071.wav
Moved: LJ010-0131.wav
Moved: LJ010-0158.wav
Moved: LJ010-0204.wav
Moved: LJ010-0312.wav
Moved: LJ011-0061.wav
Moved: LJ011-0069.wav
Moved: LJ011-0220.wav
Moved: LJ011-0254.wav
Moved: LJ012-0059.wav
Moved: LJ012-0108.wav
Moved: LJ012-0114.wav
Moved: LJ012-0264.wav
Moved: LJ013-0018.wav
Moved: LJ013-0205.wav
Moved: LJ013-0224.wav
Moved: LJ013-0229.wav
Moved: LJ013-0237.wav
Moved: LJ014-0054.wav
Moved: LJ0

## VCTK

In [12]:
#Setting source path
source="D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\datasets\VCTK-Corpus-0.92.zip"

In [13]:
vctk = transcript[transcript['Dataset'] == 'VCTK'].copy()

In [14]:
#Creating Combined (path + filename) 
# Get speaker ID from filename (before the first underscore)
vctk['Speaker'] = vctk['FileName'].str.extract(r'(p\d+)', expand=False)

# Add .flac extension
vctk['FlacName'] = vctk['FileName'] + '.flac'

# Combine full path
vctk['Combined'] = 'wav48_silence_trimmed/' + vctk['Speaker'] + '/' + vctk['FlacName']


In [19]:
#Getting List of VCTK file names
selected_files=vctk['Combined'].unique()
selected_files=list(selected_files)
# Make sure selected_files is clean: no None, no NaN, no empty strings
selected_files = [f for f in selected_files if isinstance(f, str) and f.strip() != '' and f.lower() != 'nan']


In [20]:
#Extracting the required Audio Files
int_dest='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\datasets\VCTK'

for i in range(len(selected_files)):
    with ZipFile(source, 'r') as zipObj:
        zipObj.extract(selected_files[i],int_dest)    
        zipObj.close()

In [21]:
#Moving all files from subfolders into one final directory.

source='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\datasets\VCTK\wav48_silence_trimmed'
int_destination='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\datasets\VCTK TAKEN_OUT'

for root, dirs, files in os.walk(source):  # replace the . with your starting directory
    for file in files:
      path_file = os.path.join(root,file)
      shutil.copy2(path_file,int_destination) # change you destination dir

### Converting from .flac to .wav

In [26]:
from pydub import AudioSegment

source='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\datasets\VCTK TAKEN_OUT'
int_destination='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\datasets\VCTK wavs'

In [28]:
#Getting file names
flac_files = [f for f in os.listdir(source) if f.endswith('.flac')]

for i in range(len(flac_files)):
    song=AudioSegment.from_file(source+"\\"+flac_files[i])
    song.export(int_destination+flac_files[i][:-4]+"wav",format = "wav")

### Moving to the final destination

In [29]:
source='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\datasets\VCTK wavs'

In [30]:
files=os.listdir(source)

In [32]:
for i in range(len(files)):
    shutil.copy2(source+"\\"+files[i], destination)

## IEMOCAP

In [82]:
source='iemocap/Audio Files/IEMOCAP_full_release/'
int_dest='iemocap/Audio Files/Required Wav Files/'

In [84]:
iemocap_b=pd.read_csv("Detoxy-B/Transcripts/IEMOCAP.csv")
iemocap_full=pd.read_csv("Datasets/iemocap.csv")
iemocap_full=iemocap_full[['FileName','Session']]
iemocap=pd.merge(iemocap_b,iemocap_full,on=['FileName'],how='left')

In [85]:
#Creating combined path (path + filename)

file=iemocap[['FileName','Session']].copy()
file['Session']="Session"+file['Session'].map(str)

file['path']=file['FileName']

for i in range(file.shape[0]):
    temp=file['path'].iloc[i].split('_')
    temp='_'.join(temp[:-1])
    file['path'].iloc[i]=temp
    
file['Combined path']=file['Session']+"/sentences/wav/"+file['path']+"/"+file['FileName']+'.wav'

In [86]:
selected_files=file['Combined path'].unique()
selected_files=list(selected_files)

In [87]:
#Moving audio files
for i in range(len(selected_files)):
    shutil.copy2(source+selected_files[i], int_dest)

## MSP-IMPROV

In [99]:
source='MSP-Improv/All Audio Files/'
int_dest='MSP-Improv/Required Audio Files/'

In [101]:
msp_improv=transcript[transcript['Dataset']=='MSP-Improv']
msp_improv.reset_index(drop=True,inplace=True)

In [104]:
#Creating path (path + filename)
temp=msp_improv['FileName'].str.split('-',expand=True)
temp['Session']=np.nan
for i in range(temp.shape[0]):
    temp['Session'].iloc[i]='session'+str(temp[3].iloc[i][-1:])
    
temp['Path']=temp['Session']+"/"+temp[2]+"/"+temp[4]
msp_improv['path']=temp['Path']

In [106]:
msp_improv['FileName']=msp_improv['FileName'].str.replace('.txt','.wav')
msp_improv['path']=msp_improv['path']+"/"+msp_improv['FileName']

In [107]:
selected_files=msp_improv['path'].unique()
selected_files=list(selected_files)

In [109]:
#Moving files into intermediate location
for i in range(0,len(selected_files)):
    shutil.copy2(source+selected_files[i], int_dest)

In [110]:
#Moving into Final DeToxy Location
files=os.listdir(int_dest)

for i in range(0,len(files)):
    shutil.copy2(int_dest+files[i], destination)

## Switchboard

In [111]:
source='Data/Switchboard/Audio Files/switch_board_needed/'
files=os.listdir(source)

In [112]:
switchboard=transcript[transcript['Dataset']=='Switchboard']
switchboard.reset_index(drop=True,inplace=True)

In [113]:
#Getting required filenames
selected_files=switchboard['FileName'].unique()
selected_files=list(selected_files)

In [114]:
#Moving files into final location
for i in range(0,len(selected_files)):
    shutil.copy2(source+selected_files[i]+".wav", destination)

## CMU MOSEI

In [116]:
source='Data/mosei/'
int_dest='Data/mosei/Required Audio Files/'

In [117]:
mosei_raw=pd.read_csv("Data/Detoxy-B/Transcripts/CMU-MOSEI.csv")

In [118]:
#Getting Original File names
selected_files=mosei_raw['OriginalFileName'].unique()
selected_files=list(selected_files)

In [120]:
#Extracting Audio Files
for i in range(len(selected_files)):
    with ZipFile(source+'CMU_MOSEI.zip', 'r') as zipObj:
        zipObj.extract('Raw/Audio/Full/WAV_16000/'+selected_files[i]+".wav",int_dest)    
        zipObj.close()

In [121]:
source='Data/mosei/Required Audio Files/Raw/Audio/Full/WAV_16000/'
int_dest='Data/mosei/Required Audio Files/Segmented/'

In [123]:
#Extracting Required Audio Files. 
for i in range(mosei_raw.shape[0]):
    
    old_audio = AudioSegment.from_wav(source+mosei_raw['OriginalFileName'].iloc[i]+".wav")
    new_audio=old_audio[mosei_raw['Starting'].iloc[i]*1000:mosei_raw['Ending'].iloc[i]*1000]

    new_audio.export(int_dest+mosei_raw['FileName'].iloc[i]+".wav",format = "wav",bitrate='256k')

In [124]:
#Moving into final DeToxy Folder
files=os.listdir(int_dest)

for i in range(0,len(files)):
    shutil.copy2(int_dest+files[i], destination)

## CMU - MOSI

In [125]:
source='Data/mosi/Raw/Audio/WAV_16000/Segmented/'

In [126]:
#Getting Required File names
selected_files=transcript[transcript['Dataset']=='CMU-MOSI']['FileName'].unique()
selected_files=list(selected_files)

In [127]:
#Moving into final DeToxy Folder
for i in range(len(selected_files)):
    shutil.copy2(source+selected_files[i]+".wav", destination)

## Social IQ

In [128]:
source='Data/socialiq/'
int_dest='Data/socialiq/Required Audio Files/'

In [132]:
social_iq_raw=pd.read_csv("Data/Detoxy-B/Transcripts/Social-IQ.csv")

In [ ]:
#Creating New File Names
social_iq_raw['FileName']=social_iq_raw['FileName'].str.replace('.txt','.wav')
social_iq_raw['FileName']=social_iq_raw['FileName'].str.replace('-trimmed','_trimmed-out')
social_iq_raw['File Id']=social_iq_raw.index+1

In [135]:
selected_files=social_iq_raw['FileName'].unique()
selected_files=list(selected_files)

In [ ]:
#Extracting Audio Files
for i in range(len(selected_files)):
    with ZipFile(source+'Social-IQ.zip', 'r') as zipObj:
        zipObj.extract('raw/acoustic/wav/'+selected_files[i],int_dest)    
        zipObj.close()

In [ ]:
source='Data/socialiq/Required Audio Files/raw/acoustic/wav/'
int_dest='Data/socialiq/Required Audio Files/Segmented/'

In [ ]:
#Extracting Audio Segments
for i in range(social_iq_raw.shape[0]):
    
    old_audio = AudioSegment.from_wav(source+social_iq_raw['FileName'].iloc[i])
    new_audio=old_audio[max(0,(float(social_iq_raw['start'].iloc[i].split(":")[1]) + 0.2) *1000) : (float(social_iq_raw['end'].iloc[i].split(":")[1]) + 0.2)*1000]

    new_audio.export(int_dest+social_iq_raw['FileName'].iloc[i][:-4]+str(social_iq_raw['File Id'].iloc[i])+".wav",format = "wav",bitrate='256k')

In [137]:
#Moving into final location
source='Data/socialiq/Required Audio Files/Segmented/'
selected_files=os.listdir(source)

for i in range(len(selected_files)):
    shutil.copy2(source+selected_files[i], destination)

In [139]:
social_iq_raw['FileName']=social_iq_raw['FileName'].str.replace('.wav','')

In [ ]:
social_iq_raw.to_csv("Data/Detoxy-B/Transcripts/Social-IQ.csv",index=False)

## MSP - Podcast

In [140]:
source='Data/MSP-Podcast/Audio Files/Wav 16khz files/'

In [141]:
msp_podcast=transcript[transcript['Dataset']=='MSP-Podcast']
msp_podcast.reset_index(drop=True,inplace=True)

In [142]:
selected_files=msp_podcast['FileName'].unique()
selected_files=list(selected_files)

In [145]:
for i in range(len(selected_files)):
    shutil.copy2(source+selected_files[i]+'.wav', final_destination)

## Converting to 16Khz 

In [36]:
from tqdm import tqdm
import soundfile as sf

In [37]:
files=os.listdir(destination)

In [38]:
for file in tqdm(files):
    filename = destination + file
    try:
        y, sr = librosa.load(filename, sr=16000)
        y_16 = librosa.resample(y,orig_sr=sr,target_sr=16000)
        sf.write(final_destination + file,y_16, 16000,'PCM_16')
    except:
        print(filename)
print("------------------End------------------------")

100%|██████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 1399.50it/s]

D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audioaudio
D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audioaudio1
D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audioen.tar.gz
D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audiotrain_splits
D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audiovctk
D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\AudioWav Files
D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\AudioWav-Common voice
------------------End------------------------


## Splitting audio into train,test and split

In [43]:
source='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audio\Audioo'

In [44]:
final_transcripts=pd.read_csv("D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\Final_filered_transcripts.csv",  encoding='ISO-8859-1')

In [41]:
import pandas as pd

# Try with ISO-8859-1 (also known as Latin-1) — works for many cases like this
df = pd.read_csv(r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\Transcripts.csv", encoding='ISO-8859-1')

# Filter for specific datasets (adjust capitalization as per your CSV)
filtered_df = df[df['Dataset'].isin(['MELD', 'LJ Speech', 'Common Voice', 'VCTK'])]

# Keep only the FileName column (you can add more columns if needed)
filtered_df = filtered_df[['FileName','Dataset', 'text',	'label2a',	'split',	'Sentiment',	'Original FileName',	'Starting',	'Ending'
]]

# Save to new CSV
filtered_df.to_csv(r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\Final_filered_transcripts.csv", index=False)


In [52]:
import os
import shutil

source = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audio\Audioo"
destination_train = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Final_Audio\Train"

os.makedirs(destination_train, exist_ok=True)

train = final_transcripts[final_transcripts['split'] == 0]
train.reset_index(drop=True, inplace=True)

selected_files = train['FileName'].unique().tolist()

for original_name in selected_files:
    filename = original_name  # Keep a copy of original
    real_name = filename

    # Rule 1: Remove known prefixes
    if filename.startswith(('train_', 'test_', 'dev_')):
        real_name = filename.split('_', 1)[1]  # Remove prefix only

    # Rule 2: Files starting with 'p' need to be looked up in VCTK_wavs/
    if real_name.startswith('p'):
        real_name = os.path.join('VCTK wavs'+ real_name + '.wav')
    else:
        real_name = real_name + '.wav'

    src_file = os.path.join(source, real_name)
    dest_file = os.path.join(destination_train, filename + '.wav')  # Save with original name

    try:
        shutil.copy2(src_file, dest_file)
    except FileNotFoundError:
        print(f"⚠️ Missing: {real_name}")

print("✅ All done, Princess. Your Train folder is packed and ready! 🎧")



⚠️ Missing: dia259_utt9.wav
⚠️ Missing: dia181_utt5.wav
⚠️ Missing: dia156_utt15.wav
⚠️ Missing: dia230_utt7.wav
⚠️ Missing: dia123_utt15.wav
⚠️ Missing: dia140_utt6.wav
⚠️ Missing: dia196_utt15.wav
⚠️ Missing: dia166_utt7.wav
⚠️ Missing: dia268_utt10.wav
⚠️ Missing: dia246_utt8.wav
⚠️ Missing: dia203_utt11.wav
⚠️ Missing: dia215_utt2.wav
⚠️ Missing: dia240_utt16.wav
⚠️ Missing: dia144_utt10.wav
⚠️ Missing: s5_103_mic1.wav
⚠️ Missing: dia125_utt18.wav
⚠️ Missing: dia240_utt8.wav
⚠️ Missing: dia174_utt17.wav
✅ All done, Princess. Your Train folder is packed and ready! 🎧


In [53]:
import os
import shutil

source = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audio\Audioo"
destination_test = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Final_Audio\Test"

os.makedirs(destination_test, exist_ok=True)

test = final_transcripts[final_transcripts['split'] == 2]
test.reset_index(drop=True, inplace=True)

selected_files = test['FileName'].unique().tolist()

for original_name in selected_files:
    filename = original_name  # Keep a copy of original
    real_name = filename

    # Rule 1: Remove known prefixes
    if filename.startswith(('train_', 'test_', 'dev_')):
        real_name = filename.split('_', 1)[1]  # Remove prefix only

    # Rule 2: Files starting with 'p' need to be looked up in VCTK_wavs/
    if real_name.startswith('p'):
        real_name = os.path.join('VCTK wavs'+ real_name + '.wav')
    else:
        real_name = real_name + '.wav'

    src_file = os.path.join(source, real_name)
    dest_file = os.path.join(destination_test, filename + '.wav')  # Save with original name

    try:
        shutil.copy2(src_file, dest_file)
    except FileNotFoundError:
        print(f"⚠️ Missing: {real_name}")

print("✅ Test files copied, my Sweet Shooting Star! 🌠")


⚠️ Missing: dia205_utt7.wav
⚠️ Missing: dia166_utt3.wav
⚠️ Missing: dia268_utt11.wav
⚠️ Missing: dia254_utt12.wav
⚠️ Missing: dia123_utt13.wav
⚠️ Missing: dia125_utt17.wav
✅ Test files copied, my Sweet Shooting Star! 🌠


In [54]:
import os
import shutil

source = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audio\Audioo"
destination_dev = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Final_Audio\Dev"

os.makedirs(destination_dev, exist_ok=True)

dev = final_transcripts[final_transcripts['split'] == 1]
dev.reset_index(drop=True, inplace=True)

selected_files = dev['FileName'].unique().tolist()

for original_name in selected_files:
    filename = original_name  # Keep a copy of original
    real_name = filename

    # Rule 1: Remove known prefixes
    if filename.startswith(('train_', 'test_', 'dev_')):
        real_name = filename.split('_', 1)[1]  # Remove prefix only

    # Rule 2: Files starting with 'p' need to be looked up in VCTK_wavs/
    if real_name.startswith('p'):
        real_name = os.path.join('VCTK wavs'+ real_name + '.wav')
    else:
        real_name = real_name + '.wav'

    src_file = os.path.join(source, real_name)
    dest_file = os.path.join(destination_dev, filename + '.wav')  # Save with original name

    try:
        shutil.copy2(src_file, dest_file)
    except FileNotFoundError:
        print(f"⚠️ Missing: {real_name}")

print("✅ Dev files done, Honeybee! You're unstoppable! 🐝🌈")



⚠️ Missing: dia157_utt7.wav
⚠️ Missing: dia217_utt5.wav
⚠️ Missing: dia25_utt14.wav
⚠️ Missing: dia278_utt14.wav
⚠️ Missing: dia270_utt12.wav
⚠️ Missing: dia178_utt6.wav
✅ Dev files done, Honeybee! You're unstoppable! 🐝🌈


## Creating Trigger Term Dataset

In [76]:
import pandas as pd

# Try with ISO-8859-1 (also known as Latin-1) — works for many cases like this
df = pd.read_csv(r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\trigger_test.csv", encoding='ISO-8859-1')

# Filter for specific datasets (adjust capitalization as per your CSV)
filtered_df = df[df['Dataset'].isin(['MELD', 'LJ Speech', 'Common Voice', 'VCTK'])]

# Keep only the FileName column (you can add more columns if needed)
filtered_df = filtered_df[['FileName','Dataset', 'text',	'label2a',	'Starting',	'Ending']]

# Save to new CSV
filtered_df.to_csv(r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\Final_trigger_test_transcripts.csv", index=False)

In [4]:
sourcecsv=r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\Final_trigger_test_transcripts.csv"

In [5]:
source='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audio'

In [11]:
#Opening Tar File
int_dest=source+'Audio Files - Required/'
tar = tarfile.open(source+"\\"+"en.tar.gz")

In [12]:
trigger_test_list=  pd.read_csv("D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\Final_trigger_test_transcripts.csv", encoding='ISO-8859-1')


In [15]:
#Getting List of Common Voice file names
selected_files=trigger_test_list['FileName'].unique()
selected_files=list(selected_files)

In [16]:
from tqdm import tqdm

file_names_set = set("cv-corpus-6.1-2020-12-11/en/clips/" + suit + ".mp3" for suit in selected_files)

members = [x for x in tar.getmembers() if x.name in file_names_set]

success_count = 0
fail_count = 0

for member in tqdm(members, desc="Extracting files"):
    try:
        tar.extract(member, path=int_dest)
        print(f"✅ Successfully extracted: {member.name}") 
        success_count += 1
    except FileNotFoundError:
        print(f"❌ Failed to extract (Not found): {member.name}")
        fail_count += 1
    except Exception as e:
        print(f"⚠️ Error extracting {member.name}: {e}")
        fail_count += 1

print(f"\n🎉 Extraction finished. Success: {success_count}, Failed: {fail_count}")

BadGzipFile: CRC check failed 0xa349893c != 0x13ed5fdc

In [17]:
source='D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\AudioAudio Files - Required\cv-corpus-6.1-2020-12-11\en\clips'

In [18]:
from os import path
from pydub import AudioSegment

In [ ]:
import os
import pandas as pd
from pydub import AudioSegment
import traceback

source = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\AudioAudio Files - Required\cv-corpus-6.1-2020-12-11\en\clips"
int_dest = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Final_Audio\\"

os.makedirs(int_dest + "Trigger_test_Files", exist_ok=True)

# Load only required file names from CSV
csv_path = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\Final_trigger_test_transcripts.csv"
df = pd.read_csv(csv_path)
required_files = set(df['FileName'].tolist())

files = os.listdir(source)

for i in range(len(files)):
    file_name = files[i]
    if file_name.endswith('.mp3') and file_name[:-4] in required_files:
        try:
            sound = AudioSegment.from_mp3(os.path.join(source, file_name))
            dest_path = os.path.join(int_dest, "Trigger_test_Files", file_name[:-4] + ".wav")
            sound.export(dest_path, format="wav")
        except Exception as e:
            print(f"⚠️ Skipping file due to error: {file_name}")
            print(f"   Error: {e}")
            # Optionally show traceback:
            # traceback.print_exc()

print("✅ Conversion done, Cutiebyte! Only the selected audio files were processed 🎶💖")


In [78]:
import os
import shutil
import pandas as pd

# Load the CSV
csv_path = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\Final_trigger_test_transcripts.csv"
final_transcripts = pd.read_csv(csv_path)

# Define source and destination
source = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Audio\Audioo"
destination = r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Final_Audio\trigger_test"
os.makedirs(destination, exist_ok=True)

# Use ALL file names from the CSV
selected_files = final_transcripts['FileName'].unique().tolist()

for original_name in selected_files:
    filename = original_name
    real_name = filename

    # Remove known prefixes
    if filename.startswith(('train_', 'test_', 'dev_')):
        real_name = filename.split('_', 1)[1]

    # If starts with 'p', it's in 'VCTK wavs' subdir
    if real_name.startswith('p'):
        real_name = os.path.join('VCTK wavs'+ real_name + '.wav')
    else:
        real_name = real_name + '.wav'

    # Full source and destination paths
    src_file = os.path.join(source, real_name)
    dest_file = os.path.join(destination, filename + '.wav')  # Save using original name

    try:
        shutil.copy2(src_file, dest_file)
    except FileNotFoundError:
        print(f"⚠️ Missing: {real_name}")

print("🎉 All files copied, Muffin! You're magic! ✨🦋")


⚠️ Missing: common_voice_en_22960418.wav
⚠️ Missing: common_voice_en_527188.wav
⚠️ Missing: common_voice_en_20105322.wav
⚠️ Missing: common_voice_en_71249.wav
⚠️ Missing: common_voice_en_17263212.wav
⚠️ Missing: common_voice_en_11820276.wav
⚠️ Missing: common_voice_en_676513.wav
⚠️ Missing: common_voice_en_23578064.wav
⚠️ Missing: common_voice_en_678819.wav
⚠️ Missing: common_voice_en_18495497.wav
⚠️ Missing: common_voice_en_17255320.wav
⚠️ Missing: common_voice_en_23718981.wav
⚠️ Missing: common_voice_en_20270777.wav
⚠️ Missing: common_voice_en_428103.wav
⚠️ Missing: common_voice_en_200414.wav
⚠️ Missing: common_voice_en_15758224.wav
⚠️ Missing: common_voice_en_18839962.wav
⚠️ Missing: common_voice_en_679166.wav
⚠️ Missing: common_voice_en_39020.wav
⚠️ Missing: common_voice_en_17369620.wav
⚠️ Missing: common_voice_en_17785885.wav
⚠️ Missing: common_voice_en_17283284.wav
⚠️ Missing: common_voice_en_22746572.wav
⚠️ Missing: common_voice_en_18052423.wav
⚠️ Missing: common_voice_en_178758

In [65]:
# Not including the final dataset to avoid double counting.
files=os.listdir(source)
files.remove("Transcripts.csv")

In [70]:
#Initialising a new dataframe
combined_df=pd.DataFrame()
combined_df[['Dataset','FileName','text','label2a','Trigger','Sentiment','Toxic Filter','split']]=np.nan

In [73]:
import os
import pandas as pd

combined_df = pd.DataFrame()
files = [f for f in os.listdir(source) if f.endswith('.csv') and not f.startswith('.')]

for i in range(len(files)):
    file_path = os.path.join(source, files[i])
    
    # Try reading with UTF-8, fallback to ISO-8859-1
    try:
        temp = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        temp = pd.read_csv(file_path, encoding='ISO-8859-1')
    
    # Optional: Check if all required columns are present
    required_cols = ['Dataset', 'FileName', 'text', 'Label', 'Trigger', 'Sentiment', 'Toxic Filter']
    missing_cols = [col for col in required_cols if col not in temp.columns]
    
    if not missing_cols:
        temp = temp[required_cols]
        combined_df = pd.concat([combined_df, temp], ignore_index=True)
    else:
        print(f"⚠️ Skipping {files[i]} due to missing columns: {missing_cols}")


⚠️ Skipping MELD.csv due to missing columns: ['Dataset', 'FileName', 'text', 'Label', 'Trigger', 'Toxic Filter']
⚠️ Skipping metadata.csv due to missing columns: ['FileName', 'text', 'Label', 'Trigger', 'Sentiment', 'Toxic Filter']
⚠️ Skipping test.csv due to missing columns: ['Label', 'Trigger', 'Sentiment', 'Toxic Filter']
⚠️ Skipping train.csv due to missing columns: ['Label', 'Trigger', 'Sentiment', 'Toxic Filter']
⚠️ Skipping train1.csv due to missing columns: ['Label', 'Trigger', 'Sentiment', 'Toxic Filter']
⚠️ Skipping Transcripts.csv due to missing columns: ['Label', 'Trigger', 'Toxic Filter']
⚠️ Skipping trigger_test.csv due to missing columns: ['Label', 'Trigger', 'Sentiment', 'Toxic Filter']
⚠️ Skipping valid.csv due to missing columns: ['Label', 'Trigger', 'Sentiment', 'Toxic Filter']


In [74]:
#Cleaning the data
combined_df['Trigger'].fillna(0,inplace=True)
combined_df['FileName']=combined_df['FileName'].str.replace('.wav','')
combined_df['FileName']=combined_df['FileName'].str.replace('.mp3','')
combined_df['FileName']=combined_df['FileName'].str.replace('.txt','')

KeyError: 'Trigger'

In [47]:
#Subsetting only trigger term dataset
trigger=combined_df[combined_df['Trigger']==1]
trigger.reset_index(drop=True,inplace=True)
trigger['Trigger']=trigger['Trigger'].astype(int)

In [38]:
source='Final Audio/'
destination='Data/Detoxy-B/DeToxy/Input/trigger/'

In [39]:
#Getting Required File Names
selected_files=trigger['FileName'].unique()
selected_files=list(selected_files)

In [ ]:
#Moving into final location
for i in range(len(selected_files)):
    shutil.copy2(source+selected_files[i]+'.wav', destination)

In [1]:
import pandas as pd

# Try with ISO-8859-1 (also known as Latin-1) — works for many cases like this
df = pd.read_csv(r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\train.csv", encoding='ISO-8859-1')

# Filter for specific datasets (adjust capitalization as per your CSV)
filtered_df = df[df['Dataset'].isin(['MELD', 'LJ Speech', 'Common Voice', 'VCTK'])]

# Keep only the FileName column (you can add more columns if needed)
filtered_df = filtered_df[['FileName','Dataset', 'text',	'label2a',	'Starting',	'Ending']]

# Save to new CSV
filtered_df.to_csv(r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\Final_train.csv", index=False)

In [2]:
import pandas as pd

# Try with ISO-8859-1 (also known as Latin-1) — works for many cases like this
df = pd.read_csv(r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\test.csv", encoding='ISO-8859-1')

# Filter for specific datasets (adjust capitalization as per your CSV)
filtered_df = df[df['Dataset'].isin(['MELD', 'LJ Speech', 'Common Voice', 'VCTK'])]

# Keep only the FileName column (you can add more columns if needed)
filtered_df = filtered_df[['FileName','Dataset', 'text',	'label2a',	'Starting',	'Ending']]

# Save to new CSV
filtered_df.to_csv(r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\Final_test.csv", index=False)

In [3]:
import pandas as pd

# Try with ISO-8859-1 (also known as Latin-1) — works for many cases like this
df = pd.read_csv(r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\valid.csv", encoding='ISO-8859-1')

# Filter for specific datasets (adjust capitalization as per your CSV)
filtered_df = df[df['Dataset'].isin(['MELD', 'LJ Speech', 'Common Voice', 'VCTK'])]

# Keep only the FileName column (you can add more columns if needed)
filtered_df = filtered_df[['FileName','Dataset', 'text',	'label2a',	'Starting',	'Ending']]

# Save to new CSV
filtered_df.to_csv(r"D:\Toxicity-Detection-in-Spoken-Utterances\Toxicity-Detection-in-Spoken-Utterances\Data\Final_valid.csv", index=False)